# Objetivo:

    You must predict the total count of bikes rented during each hour covered by the test set,
    using only information available prior to the rental period

In [ ]:
!pip install feature-engine

In [ ]:
import pandas as pd
import numpy as np
import missingno as msg
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore',category=UserWarning)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

# Importação de bases

In [ ]:
# Treino
df = pd.read_csv("/content/drive/MyDrive/datasets/mentoria/bike-sharing-demand/train.csv",
                 sep=',',
                 encoding='UTF-8',
                 parse_dates=['datetime'])

# Criando cópia
df_train = df.copy()

In [ ]:
df_train.head()

In [ ]:
# Linhas e colunas
df_train.shape

In [ ]:
# Tipos das colunas
df_train.dtypes

## Observações:

**Weather:**

1.   Clear, Few clouds, Partly cloudy, Partly cloudy
2.   Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3.   Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

**Season:**
1 = spring, 2 = summer, 3 = fall, 4 = winter

# Analisando a base

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.nunique()

In [ ]:
# Missingno: sem nulos nas colunas
msg.bar(df_train, figsize=(10,5), fontsize=12);

In [ ]:
# Espaço de tempo da base (2 anos)
df_train.agg({'datetime':['min','max']})

# Análise Exploratória

In [ ]:
# Criando variáveis de tempo
df_train['date']    = df_train['datetime'].apply(lambda x: x.date())
df_train['year']    = df_train['datetime'].apply(lambda x: x.year)
df_train['month']   = df_train['datetime'].apply(lambda x: x.month)
df_train['day']     = df_train['datetime'].apply(lambda x: x.day)
df_train['weekday'] = df_train['datetime'].apply(lambda x: x.weekday())
df_train['hour']    = df_train['datetime'].apply(lambda x: x.hour)

In [ ]:
# Ajustando o tipo das colunas
cat_vars = ['season','weather','date','year','month','day','weekday']

df_train[cat_vars] = df_train[cat_vars].astype('category')

In [ ]:
# Correlação
df_corr = df_train[["temp","atemp","casual","registered","humidity","windspeed","count", "hour","weekday"]].corr()

mask = np.array(df_corr)
mask[np.tril_indices_from(mask)] = False

plt.figure(figsize=(10,5))
sns.heatmap(df_corr, mask=mask, annot=True)
plt.show()

In [ ]:
# Estatísticas descritivas
df_train.describe(percentiles=[0.1,0.25,0.5,0.75,0.9]).round(2)

In [ ]:
# Coeficiente de Variação
df_train.std()/df_train.mean()

## Tabelas com agrupamento de variáveis temporais

In [ ]:
# Comparação Ano
df_train.groupby('year').mean().round(2)

In [ ]:
# Comparação Mês
df_train.groupby('month').mean().round(2)

In [ ]:
# Comparação dia da semana
df_train.groupby('weekday').mean().round(2)

In [ ]:
# Comparação Hora
df_train.groupby('hour').mean().round(2)

In [ ]:
# Dataframe com valores por dia
df_timeseries = (
    df_train.groupby('date')
    .agg(
        soma_dia   = ('count', 'sum'),
        media_hora = ('count','mean'),
        dp         = ('count','std')
        )
    .assign(media_plus1dp  = lambda df: df.media_hora+df.dp)
    .assign(media_minus1dp = lambda df: df.media_hora-(df.dp))
    .assign(mediamovel_5d  = lambda df: df.media_hora.rolling(5).mean())
    .assign(mediamovel_10d = lambda df: df.media_hora.rolling(10).mean())
    .assign(mediamovel_30d = lambda df: df.media_hora.rolling(30).mean())
    .drop(['soma_dia','dp'], axis=1)
)

df_timeseries.head()

## Tabelas com agrupamento de variáveis categóricas

In [ ]:
# Comparação dia da semana
df_train.groupby('season').mean().round(2)

In [ ]:
# Comparação dia da semana
df_train.groupby('workingday').mean().round(2)

In [ ]:
# Comparação dia da semana
df_train.groupby('weather').mean().round(2)

In [ ]:
# Comparação dia da semana
df_train.groupby('holiday').mean().round(2)

In [ ]:
# Estatísticas do número de bikes alugadas por temporada
df_train.groupby('season').agg(
    {
        'count': ['count','mean','std','max','min']
    }
)

In [ ]:
# Count = casual + registered
# Removendo colunas que não serão utilizadas.
df_train.drop(['casual','registered','datetime'], axis=1, inplace=True)

## Plotagem

In [ ]:
# Linha do tempo
fig,axs = plt.subplots(2,1,figsize=(25,15))

color_palette = 'inferno'

cols1 = ['media_hora','media_plus1dp','media_minus1dp']
cols2 = ['media_hora','mediamovel_5d','mediamovel_10d','mediamovel_30d']

sns.lineplot(data=df_timeseries[cols1], palette=color_palette, ci=None, ax=axs[0])
axs[0].set_title('Série temporal - Quantidade de Bikes alugadas', fontsize=20)
axs[0].set_xlabel(None)
axs[0].grid(axis='y', linestyle='--', linewidth=0.5)

sns.lineplot(data=df_timeseries[cols2], palette=color_palette, ci=None, ax=axs[1])
axs[1].set_title('Média Móvel - Quantidade de Bikes alugadas', fontsize=20)
axs[1].set_xlabel(None)
axs[1].grid(axis='y', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(25,10))
plt.subplots_adjust(hspace=0.25)

#histograma
sns.barplot(data=df_train, x='year', y='count', palette='inferno', ax=axs[0,0])
sns.barplot(data=df_train, x='month', y='count', palette='inferno', ax=axs[0,1])
sns.barplot(data=df_train, x='weekday', y='count', palette='inferno', ax=axs[0,2])
sns.barplot(data=df_train, x='hour', y='count', palette='inferno', ax=axs[0,3])

axs[0,0].set_title('Bikes alugadas por ano', fontsize=15)
axs[0,1].set_title('Bikes alugadas por mês', fontsize=15)
axs[0,2].set_title('Bikes alugadas por dia da semana', fontsize=15)
axs[0,3].set_title('Bikes alugadas por hora do dia', fontsize=15)

#boxplot
sns.boxplot(data=df_train, x='year', y='count', palette='inferno', ax=axs[1,0])
sns.boxplot(data=df_train, x='month', y='count', palette='inferno', ax=axs[1,1])
sns.boxplot(data=df_train, x='weekday', y='count', palette='inferno', ax=axs[1,2])
sns.boxplot(data=df_train, x='hour', y='count', palette='inferno', ax=axs[1,3])

axs[1,0].set_title('Bikes alugadas por ano', fontsize=15)
axs[1,1].set_title('Bikes alugadas por mês', fontsize=15)
axs[1,2].set_title('Bikes alugadas por dia da semana', fontsize=15)
axs[1,3].set_title('Bikes alugadas por hora do dia', fontsize=15)

plt.show()

In [ ]:
# Distribuição de probabilidade
fig, axs = plt.subplots(1,3, figsize=(25,5))

sns.histplot(data=df_train, x='count', binwidth=20, ax=axs[0], color='darkorange')
sns.kdeplot(data=df_train, x='count', fill=True, bw_adjust=0.5, ax=axs[1], color='darkorange')
sns.ecdfplot(data=df_train, x='count', ax=axs[2], color='darkorange')

axs[0].set_title('Histograma - Quantidade de Bike Alugadas por Hora', fontsize=15)
axs[0].set_xlabel(None)
axs[1].set_title('Estimativa de Densidade de Kernel', fontsize=15)
axs[1].set_xlabel(None)
axs[2].set_title('Proporção cumulativa %', fontsize=15)
axs[2].set_xlabel(None)
axs[2].grid()

plt.show()

In [ ]:
#Boxplot
fig, axs = plt.subplots(2,2, figsize=(15,10))
plt.subplots_adjust(hspace=0.25)

sns.boxplot(data=df_train, x='season', y='count', palette='inferno', ax=axs[0,0])
axs[0,0].set_title('Boxplot - Season', fontsize=15)
axs[0,0].set_xlabel(None)

sns.boxplot(data=df_train, x='holiday', y='count', palette='inferno', ax=axs[0,1])
axs[0,1].set_title('Boxplot - holiday', fontsize=15)
axs[0,1].set_xlabel(None)

sns.boxplot(data=df_train, x='workingday', y='count', palette='inferno', ax=axs[1,0])
axs[1,0].set_title('Boxplot - workingday', fontsize=15)
axs[1,0].set_xlabel(None)

sns.boxplot(data=df_train, x='weather', y='count', palette='inferno', ax=axs[1,1])
axs[1,1].set_title('Boxplot - weather', fontsize=15)
axs[1,1].set_xlabel(None)

plt.show()

In [ ]:
# Distribuição por season
sns.displot(data=df_train,
            x='count',
            kind='hist',
            stat='probability',
            binwidth=50,
            col='season',
            height=5,
            aspect=1,
            color='darkorange')

plt.show()

In [ ]:
# KDE - Bikes rented by season
sns.displot(data=df_train,
            x='count',
            palette='inferno',
            hue='season',
            kind='kde',
            bw_adjust=0.50,
            height=8,
            aspect=2)

plt.title('Estimativa de densidade de Kernel por temporada', fontsize=15)
plt.show()

In [ ]:
# KDE - Bikes rented by weather
sns.displot(data=df_train,
            x='count',
            palette='inferno',
            hue='weather',
            kind='kde',
            bw_adjust=0.50,
            height=8,
            aspect=2)

plt.title('Estimativa de densidade de Kernel por clima', fontsize=15)
plt.show()

## Conclusões

    1. Variável com maior variabilidade é a "count"
    2. Ano de 2012 foi em média, mais quente/mais seco e com maior número de bikes alugadas
    3. Mês de Janeiro com menor número médio de aluguéis e Junho com o maior número
    4. Os picos de aluguéis estão às 8h, 17h e 18h
    5. Season 3 tem em média um maior número de bikes alugadas enquanto a season1 apresenta o menor número. Com concentração maior em quantidades menor (<50) puxando a média
    6. O maior valor registrado de bikes alugadas em uma hora foi em 2012-09-12 (season3) às 18h com 977 aluguéis
    7. Horários de pico tem menos outliers que os demais horários ao longo do dia



Observações:

Weather:

    Clear, Few clouds, Partly cloudy, Partly cloudy
    Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

Season: 1 = spring, 2 = summer, 3 = fall, 4 = winter

# Treinar Modelos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import RareLabelEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import make_scorer

In [ ]:
# Features que não ajudam na previsão
abt = df_train.drop(['date','year','day'], axis=1)

In [ ]:
# Separando features
target = 'count'
cat_vars = abt.select_dtypes(include='category').columns.tolist() # Categóricas
num_vars = abt.drop(cat_vars, axis=1).drop(target, axis=1).columns.tolist() # Numéricas

# Base de features
X = abt.drop('count', axis=1).copy()

# Valores da target
y = abt['count'].copy()

log_y = np.log1p(y)

In [ ]:
# Funções para calcular e salvar as métricas
def adjusted_r2_scorer(y_true, y_pred, features):

    r2 = r2_score(y_true, y_pred)
    N = len(y_true)
    p = features.shape[1]

    r2_ajustado = (1 - ((1 - r2) * (N - 1)) / (N - p - 1))

    return r2_ajustado

def mape_error(y_true, y_pred):

    return np.mean(np.abs((y_true - y_pred)/y_true))

def rmse_error(y_true, y_pred):

    return np.sqrt(mean_squared_error(y_true, y_pred))

def rmsle_error(y_true, y_pred):

    # transformação exponencial
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)

    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# Função pra limpar os valores do dicionário
def pop_values_dict(dict_data,n_last_values=1):

    while n_last_values > 0:   

        for _,v in enumerate(dict_data.items()): 
            
            dict_data[v[0]].pop()
        
        n_last_values -= 1

## Calculando as métricas

In [ ]:
# Lista com os modelos para rodar
models = [
    ('lr', LinearRegression()),
    ('ridge', Ridge(random_state=42)),
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42)),
    ('lgbm', LGBMRegressor(random_state=42)),
    ('xgb', XGBRegressor(random_state=42, objective='reg:squarederror'))
]

# Métricas
metrics = {
    'model': [], 'r2':    [], 'r2_ajustado': [],
    'mse':    [], 'rmse':  [], 'rmsle': [],
    'mae':    [], 'medae': [], 'mape': []
}

score_list = {
    'r2'   : make_scorer(r2_score),
    'r2_ajustado': make_scorer(adjusted_r2_scorer, features=X),
    'mse'  : make_scorer(mean_squared_error, greater_is_better=False),
    'mae'  : make_scorer(mean_absolute_error, greater_is_better=False),
    'medae': make_scorer(median_absolute_error, greater_is_better=False),
    'rmse' : make_scorer(rmse_error, greater_is_better=False),
    'rmsle': make_scorer(rmsle_error, greater_is_better=False),
    'mape' : make_scorer(mape_error, greater_is_better=False)
}

def save_metrics(X_test, y_test, y_pred, nome_modelo):

    metrics['model'].append(nome_modelo)
    metrics['r2'].append(r2_score(y_test, y_pred))
    metrics['r2_ajustado'].append(adjusted_r2_scorer(y_test, y_pred, features=X_test))
    metrics['mse'].append(mean_squared_error(y_test, y_pred))
    metrics['rmse'].append(rmse_error(y_test, y_pred))
    metrics['rmsle'].append(rmsle_error(y_test, y_pred,))
    metrics['mae'].append(mean_absolute_error(y_test, y_pred))
    metrics['medae'].append(median_absolute_error(y_test, y_pred))
    metrics['mape'].append(mape_error(y_test, y_pred),)

def save_metrics_crossval(modelo, nome_modelo, X, y, nsplits=5):

    # validação cruzada
    kf = KFold(n_splits=nsplits, shuffle=True, random_state=42)

    cv_results = cross_validate(modelo, X, y, scoring=score_list, cv=kf)

    cv_results_df = pd.DataFrame(cv_results).mean().round(5)

    # Salvando métricas
    metrics['model'].append(nome_modelo)
    metrics['r2'].append(cv_results_df['test_r2'])
    metrics['r2_ajustado'].append(cv_results_df['test_r2_ajustado'])
    metrics['mae'].append(cv_results_df['test_mae']*(-1))
    metrics['mse'].append(cv_results_df['test_mse']*(-1))
    metrics['rmse'].append(cv_results_df['test_rmse']*(-1))
    metrics['rmsle'].append(cv_results_df['test_rmsle']*(-1))
    metrics['medae'].append(cv_results_df['test_medae']*(-1))
    metrics['mape'].append(cv_results_df['test_mape']*(-1))

In [ ]:
# Rodando os modelos

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

log_y_train = np.log1p(y_train) # log(y+1)

log_y_test = np.log1p(y_test)

for model in models:

    # Pipeline
    pipe = Pipeline(
    steps=[
           ('standardscaling', SklearnTransformerWrapper(MinMaxScaler(), variables=num_vars)),
           ('rarelabelencoder', RareLabelEncoder(tol=0.01, variables=cat_vars)),
           ('onehotencoder',OneHotEncoder(drop_last=True, variables=cat_vars))
        ])
    
    # Adicionar modelo no pipeline
    pipe.steps.append(model)
    
    # Se for um desses modelos, precisa fazer normalização das features
    if model[0] in ['lr','ridge']:

        pipe.fit(X_train,log_y_train) # todos os steps
        y_pred = pipe.predict(X_test)

        save_metrics(X_test, log_y_test, y_pred, model[0])

    else:

        pipe[1:].fit(X_train,log_y_train) # todos steps menos o primeiro
        y_pred = pipe[1:].predict(X_test)

        save_metrics(X_test, log_y_test, y_pred, model[0])

pd.DataFrame(metrics)

## Cross Validation

In [ ]:
# Limpando dados das métricas
pop_values_dict(metrics,6)

In [ ]:
# Rodando os modelos
for model in models:

    # Pipeline
    pipe = Pipeline(
    steps=[
           ('standardscaling', SklearnTransformerWrapper(MinMaxScaler(), variables=num_vars)),
           ('rarelabelencoder', RareLabelEncoder(tol=0.01, variables=cat_vars)), # Não é necessário
           ('onehotencoder',OneHotEncoder(drop_last=True, variables=cat_vars))
        ])
    
    # Adicionar modelo no pipeline
    pipe.steps.append(model)

    # Cross validation KFold
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Se for um desses modelos, precisa fazer normalização das features
    if model[0] in ['lr','ridge']:

        save_metrics_crossval(pipe, model[0], X, log_y, nsplits=10)

    else:

        save_metrics_crossval(pipe[1:], model[0], X, log_y, nsplits=10)

pd.DataFrame(metrics)

**Vitória unânime do LightGBM**

Maior R² ajustado e menor RMSLE/MAE entre todos


## Otimização de parâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo pipeline do modelo
lgbm = Pipeline(
    steps=[
       ('rarelabelencoder', RareLabelEncoder(tol=0.01, variables=cat_vars)), # Não é necessário
       ('onehotencoder',OneHotEncoder(drop_last=True, variables=cat_vars)),
       ('model', LGBMRegressor(random_state=42))
       ]
    )

#Definindo parâmetros para teste
lgbm_params = {
    'model__n_estimators': [500,600,700],
    'model__num_leaves':[8,10],
    'model__max_depth':[3,4,5],
    'model__learning_rate':[0.01,0.05,0.1],
    'model__min_data_in_leaf': [20,30,40]
}

kf=KFold(5, shuffle=True, random_state=42)

gs_lgbm = GridSearchCV(lgbm, lgbm_params, scoring=score_list['rmsle'], cv=kf)

gs_lgbm.fit(X_train,log_y_train)

In [ ]:
save_metrics_crossval(gs_lgbm.best_estimator_,'lgbm_tuned_test',X, log_y,nsplits=10)

In [ ]:
gs_lgbm.best_estimator_

# Prevendo valores com o modelo otimizado.

In [ ]:
# Teste
df2 = pd.read_csv("/content/drive/MyDrive/datasets/mentoria/bike-sharing-demand/test.csv",
                 sep=',',
                 encoding='UTF-8',
                 parse_dates=['datetime'])

# Criando cópia
df_test = df2.copy()

# Variáveis de tempo
df_test['month']   = df_test['datetime'].apply(lambda x: x.month)
df_test['weekday'] = df_test['datetime'].apply(lambda x: x.weekday())
df_test['hour']    = df_test['datetime'].apply(lambda x: x.hour)

# Ajustando o tipo das colunas
cat_vars = ['season','weather','month','weekday']
df_test[cat_vars] = df_test[cat_vars].astype('category')

# Pegando apenas as features necessárias
X_test = df_test.drop('datetime', axis=1)

# Previsões (log)
y_pred = gs_lgbm.best_estimator_.predict(X_test)

# Convertendo o valor log para o real
df_test['pred_bike_rents'] = np.expm1(y_pred)

In [ ]:
# Salvando as previsões para o Kaggle
submission = pd.DataFrame({
        "datetime": df_test['datetime'],
        "count": [max(0, x) for x in df_test['pred_bike_rents']]
    })
submission.to_csv('/content/drive/MyDrive/datasets/mentoria/bike-sharing-demand/submission.csv', index=False)